# Practice 4-1: Pandas 🐼



## Part 0. Setup Steps

- Create a repo on GitHub named `eds217-practice-4-1-pandas`
- Clone to create a version-controlled project
- Create some subfolder infrastructure (nbs, data, figs)
- Create and save a new ~~quarto in RStudio called~~ jupyter notebook (`.ipynb` file) named `practice_4-1_pandas.ipynb` in the `nbs` folder.
- Open the notebook in VSCode or jupyter notebook/lab 
- Make sure to associate the notebook with the `eds217_2023` environment.

- Create a cell that imports your essential data science libraries.


## 📚  <b> Practice 0. </b> 
    
- Create a cell that imports your essential data science libraries.(`numpy` as `np`, `matplotlib.pyplot` as `plt` and `pandas` as `pd`)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 



## 📚  <b> Practice 1. </b> 

    

## Import the data for our practice session

In [2]:
# Import data
bsrn = pd.read_csv('../../data/BSRN_GOB_2019-10.csv')

## 📚  Practice 2. Using the DataFrame <code>bsrn</code>:
<ol>
    <li type="a"> Print a list of column names. </li>
    <li type="a"> How many values are there in the entire DataFrame? </li>
    <li type="a"> What is the data type of the first column? </li>
</ol>

In [3]:
# 2a. Print a list of column names
print(list(bsrn.columns))

# 2b. Values in entire DataFrame
print(bsrn.size)

# 2c. Retrieve data type of first column
print(list(bsrn.dtypes)[0]) 

['DATE', 'H_m', 'SWD_Wm2', 'STD_SWD', 'DIR_Wm2', 'STD_DIR', 'DIF_Wm2', 'STD_DIF', 'LWD_Wm2', 'STD_LWD', 'SWU_Wm2', 'LWU_Wm2', 'T_degC', 'RH', 'P_hPa']
669600
object



## 📚  Practice 3.
<ol class="alpha">
    <li> Create a new DataFrame containing the first record for each day and the following columns: the timestamp of the record, incoming shortwave radiation, direct and diffuse radiation, and incoming longwave radiation. (Hint: the BSRN station collects data every minute). </li>
    <li> Create a new Series containing the temperature values every hour at the top of the hour. </li>
</ol>


In [4]:
# Daily radiation data
bsrn_day = bsrn.iloc[::1440,:9:2]

# Hourly temperatures
temp_hr = bsrn.T_degC[::60]

In [5]:
# Convert bsrn.DATE column to datetime objects
bsrn['DATE'] = pd.to_datetime(bsrn.DATE)  # Note: overwriting a column like this is NOT recommended.

# Set bsrn.DATE as the DataFrame index
bsrn.set_index('DATE',inplace=True)

## 📚  <b> Practice 4. 
</b> Calculate the mean incoming shortwave, outgoing shortwave, incoming longwave, and outgoing longwave radiation over the entire month.

In [6]:
# 4. Mean SW in, SW out, LW in, LW out
print('Mean SW incoming radiation = ', bsrn.SWD_Wm2.mean(), ' W m-2')
print('Mean SW outgoing radiation = ', bsrn.SWU_Wm2.mean(), ' W m-2')
print('Mean LW incoming radiation = ', bsrn.LWD_Wm2.mean(), ' W m-2')
print('Mean LW outgoing radiation = ', bsrn.LWU_Wm2.mean(), ' W m-2')

Mean SW incoming radiation =  318.0465157965494  W m-2
Mean SW outgoing radiation =  110.44500448028674  W m-2
Mean LW incoming radiation =  342.35069187467764  W m-2
Mean LW outgoing radiation =  455.0540322580645  W m-2


## 📚  <b> Practice 5. </b>
<ol>
    <li type="a"> Add a column <code style='font-weight:normal'>'NET_SW'</code> to <code style='font-weight:normal'>bsrn</code> with the net shortwave radiation. </li>
    <li type="a"> Add a column <code style='font-weight:normal'>'NET_LW'</code> to <code style='font-weight:normal'>bsrn</code> with the net longwave radiation. </li>
    <li type="a"> Add a column <code style='font-weight:normal'>'NET_RAD'</code> to <code style='font-weight:normal'>bsrn</code> with the net total radiation. 
    
Net radiation is given by the following equation: </li>

$$R^{}_{N} \, = \,  R^{\, \downarrow}_{SW} \, - \,  R^{\, \uparrow}_{SW} \, + \, R^{\, \downarrow}_{LW} \, - \,  R^{\, \uparrow}_{LW}$$
    
where $R^{\, \downarrow}_{SW}$ and $R^{\, \uparrow}_{SW}$ are incoming and outgoing shortwave radiation, respectively, and $R^{\, \downarrow}_{LW}$ and $R^{\, \uparrow}_{LW}$ are incoming and outgoing longwave radiation, respectively.


<li type="a"> Create a new DataFrame with the day of the month and daily mean values of shortwave incoming, shortwave outgoing, longwave incoming, longwave outgoing radiation, and net total radiation. (Hint: use masking!).</li>
</ol>

In [7]:
# 5a. Net SW radiation
bsrn['NET_SW'] = bsrn.SWD_Wm2 - bsrn.SWU_Wm2

# 5b. Net LW radiation
bsrn['NET_LW'] = bsrn.LWD_Wm2 - bsrn.LWU_Wm2

# 5c. Net total radiation
bsrn['NET_RAD'] = bsrn.SWD_Wm2 - bsrn.SWU_Wm2 + bsrn.LWD_Wm2 - bsrn.LWU_Wm2


# 5d. Mean daily SW in, SW out, LW in, LW out, R_net
# Initialize empty list
daily_rad_list = []
# Iterate through unique day values
for d in bsrn.index.day.unique():
    # SW in
    swd = bsrn.SWD_Wm2[bsrn.index.day == d].mean()
    # SW out
    swu = bsrn.SWU_Wm2[bsrn.index.day == d].mean()
    # LW in
    lwd = bsrn.LWD_Wm2[bsrn.index.day == d].mean()
    # LW out
    lwu = bsrn.LWU_Wm2[bsrn.index.day == d].mean()
    # Net total
    rnet = swd - swu + lwd - lwu
    # Add all values to a list of lists
    daily_rad_list.append([d, swd, swu, lwd, lwu, rnet])

# Convert to DataFrame
daily_rad = pd.DataFrame(daily_rad_list, columns=['DAY', 'SW_in', 'SW_out', 'LW_in', 'LW_out', 'R_NET'])
daily_rad

DAY       SW_in      SW_out       LW_in      LW_out       R_NET
0     1  325.336345  115.334028  321.219903  458.834028   72.388192
1     2  322.732453  113.939583  318.131341  448.486111   78.438100
2     3  298.635417  103.261806  330.516667  429.979861   95.910417
3     4  323.209173  113.222222  314.129526  439.990972   84.125505
4     5  296.861806  105.461806  324.660876  427.359722   88.701153
5     6  292.118915  104.219444  330.458333  428.665278   89.692526
6     7  339.848611  119.959722  303.437413  439.002778   84.323524
7     8  332.009028  116.752778  313.126653  438.550000   89.832903
8     9  290.577083  104.436806  342.506267  428.110417  100.536129
9    10  310.887500  108.638889  331.667130  434.941667   98.974074
10   11  312.226389  108.406250  350.486787  458.562500   95.744426
11   12  313.099306  108.276389  375.387500  483.829167   96.381250
12   13  292.540972   97.745833  389.507307  485.947222   98.355224
13   14  210.243056   76.329167  390.913829  470.870139   53.957579
14   15  294.886806   91.408333  399.457639  500.201389  102.734722
15   16  322.011111  114.262500  400.745833  510.212500   98.281944
16   17  306.800000  106.773611  390.968033  492.745833   98.248589
17   18  298.656250  104.536111  364.097222  459.456944   98.760417
18   19  314.968750  110.327083  365.431250  460.609028  109.463889
19   20  329.323611  113.339583  357.791522  473.039583  100.735966
20   21  338.721334  117.433333  340.713189  470.612500   91.388690
21   22  338.259722  117.554861  325.689368  441.588194  104.806034
22   23  354.881944  122.917361  306.553857  438.565972   99.952468
23   24  344.563194  119.732639  319.499653  452.019444   92.310764
24   25  261.228472   87.575000  344.902643  441.439583   77.116531
25   26  345.552778  118.828472  337.203614  459.212500  104.715419
26   27  356.714583  123.323611  309.846421  434.491667  108.745727
27   28  358.297636  125.206944  303.482615  421.080556  115.492750
28   29  352.906944  122.894444  322.271399  448.573611  103.710288
29   30  357.626129  123.736806  331.607516  462.778472  102.718367
30   31  323.794302  107.959722  356.192629  466.917361  105.109847

In [8]:
# 7a. Construct df1 from list of lists
df1 = pd.DataFrame([['Amazon', 6400, 7050000], 
                    ['Congo', 4371, 4014500],
                    ['Yangtze', 6418, 1808500],
                    ['Mississippi', 3730, 3202230]],
                   columns=['RIVER', 'LENGTH_km', 'DRAINAGE_AREA_km2'])

# 7b. Construct df2 from dict
df2 = pd.DataFrame({'RIVER': ['Zambezi', 'Mekong', 'Murray', 'Rhone', 'Cubango'],
                    'LENGTH_km': [2574, 4023, 2508, 813, 1056],
                    'DRAINAGE_AREA_km2': [1331000, 811000, 1061469, 98000, 530000]})

## 📚  <b> Practice 6. </b> 
<ol class="alpha">
    <li> Concatenate <code>df1</code> and <code>df2</code> into a new DataFrame with all 9 rivers.</li>
    <li> Create a new DataFrame <code>rivers</code> with the discharge, mouth, source, and continent information and add this to your DataFrame from (a) to produce a DataFrame with all of the data in the table below.</li>
</ol>

| River | Length $(\text{km})$ | Drainage area $(\text{km}^2)$ | Discharge $(\text{m}^2/\text{s})$ | Mouth | Source | Continent |
|-------|----------------------|------------------------------|----------------------------------|-------|--------|-----------|
| Amazon | 6400 | 7,050,000 | 209,000 | Atlantic Ocean | Rio Mantaro | South America |
| Congo | 4371 | 4,014,500 | 41,200 | Atlantic Ocean | Lualaba River | Africa |
| Yangtze | 6418 | 1,808,500 | 30,166 | East China Sea | Jianggendiru Glacier | Asia |
| Mississippi | 3730 | 3,202,230 | 16,792 | Gulf of Mexico | Lake Itasca | North America |
| Zambezi | 2574 | 1,331,000 | 3,400 | Indian Ocean | Miombo Woodlands | Africa |
| Mekong | 4023 | 811,000 | 16,000 | South China Sea | Lasagongma Spring | Asia |
| Murray | 2508 | 1,061,469 | 767 | Southern Ocean | Australian Alps | Oceania |
| Rhône | 813 | 98,000 | 1,710 | Mediterranean Sea | Rhône Glacier | Europe |
| Cubango | 1056 | 530,000 | 475 | Okavango Delta | Bié Plateau | Africa |



In [9]:
# 6a. Concatenate df1 and df2
rivs = pd.concat([df1,df2],ignore_index=True)

# 6b. Concatenate rivs and df3
# Add new data to DataFrame
df3 = pd.DataFrame([[209000, 'Atlantic Ocean', 'Rio Mantaro', 'South America'], 
                    [41200, 'Atlantic Ocean', 'Lualaba River', 'Africa'],
                    [30166, 'East China Sea', 'Jianggendiru Glacier', 'Asia'],
                    [16792, 'Gulf of Mexico', 'Lake Itasca', 'North America'],
                    [3400, 'Indian Ocean', 'Miombo Woodlands', 'Africa'],
                    [16000, 'South China Sea', 'Lasagongma Spring', 'Asia'],
                    [767, 'Southern Ocean', 'Australian Alps', 'Oceania'],
                    [1710, 'Mediterranean Sea', 'Rhone Glacier', 'Europe'],
                    [475, 'Okavango Delta', 'Bie Plateau', 'Africa']],
                  columns=['DISCHARGE_m3s', 'MOUTH', 'SOURCE', 'CONTINENT'] )
# Concatenate rivs + df3
rivers = pd.concat([rivs,df3], axis=1)
rivers

RIVER  LENGTH_km  DRAINAGE_AREA_km2  DISCHARGE_m3s  \
0       Amazon       6400            7050000         209000   
1        Congo       4371            4014500          41200   
2      Yangtze       6418            1808500          30166   
3  Mississippi       3730            3202230          16792   
4      Zambezi       2574            1331000           3400   
5       Mekong       4023             811000          16000   
6       Murray       2508            1061469            767   
7        Rhone        813              98000           1710   
8      Cubango       1056             530000            475   

               MOUTH                SOURCE      CONTINENT  
0     Atlantic Ocean           Rio Mantaro  South America  
1     Atlantic Ocean         Lualaba River         Africa  
2     East China Sea  Jianggendiru Glacier           Asia  
3     Gulf of Mexico           Lake Itasca  North America  
4       Indian Ocean      Miombo Woodlands         Africa  
5    South China Sea     Lasagongma Spring           Asia  
6     Southern Ocean       Australian Alps        Oceania  
7  Mediterranean Sea         Rhone Glacier         Europe  
8     Okavango Delta           Bie Plateau         Africa

##  📚  <b> Practice 7. </b> 
Export your <code>rivers</code> DataFrame to a CSV file in the <code>exports</code> folder. 

In [10]:
# 7. Export rivers
rivers.to_csv('../exports/rivers.csv') # note: will be different from the practice session folder.
# rivers.to_csv('exports/rivers.csv') # note: will be different from the practice session folder.

<hr style="border-top: 1px solid gray; margin-top: 24px; margin-bottom: 1px"></hr>